In [ ]:
!pip install pybaseball

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 39.7 MB/s eta 0:00:00


In [ ]:
from pybaseball import statcast
import pandas as pd
import random


start_dt = '2022-05-07'
end_dt = '2022-05-08'

data = statcast(start_dt, end_dt)

This is a large query, it may take a moment to complete


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/usr/local/lib/python3.10/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/usr/local/lib/python3.10/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = da

In [ ]:
type(data)

pandas.core.frame.DataFrame

In [ ]:
print(data.columns.to_list())



['pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'description', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 'iso_value', 'launch_speed_a

In [ ]:
play_by_play_columns = [
    'game_date', 'inning', 'inning_topbot', 'player_name', ''
    'events', 'description', 'outs_when_up', 'balls', 'strikes',
    'bat_score', 'fld_score'
]


play_by_play = data[play_by_play_columns]
play_by_play_sorted = play_by_play.sort_values(
    by=['game_date', 'inning', 'inning_topbot', 'outs_when_up', 'balls', 'strikes'],
    ascending=[True, True, True, True, True, True]
)

play_by_play_sorted

,game_date,inning,inning_topbot,player_name,events,description,outs_when_up,balls,strikes,bat_score,fld_score
3507,2022-05-07,1,Bot,"Gray, Josiah",field_out,hit_into_play,0,0,0,0,1
4429,2022-05-07,1,Bot,"Gray, Josiah",NaN,ball,0,0,0,0,1
4443,2022-05-07,1,Bot,"Freeland, Kyle",NaN,called_strike,0,0,0,0,0
4430,2022-05-07,1,Bot,"Cease, Dylan",NaN,called_strike,0,0,0,0,0
4407,2022-05-07,1,Bot,"Kershaw, Clayton",NaN,swinging_strike,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
3800,2022-05-08,10,Top,"Sewald, Paul",NaN,swinging_strike,2,0,1,1,1
3769,2022-05-08,10,Top,"Sewald, Paul",NaN,ball,2,0,2,1,1
3613,2022-05-08,10,Top,"Sewald, Paul",NaN,ball,2,1,2,1,1
3480,2022-05-08,10,Top,"Sewald, Paul",NaN,blocked_ball,2,2,2,1,1


In [ ]:

def create_lag_features(df, lag=3):
    for i in range(1, lag + 1):
        df[f'pitch_type_lag_{i}'] = df['pitch_type'].shift(i)
        df[f'release_speed_lag_{i}'] = df['release_speed'].shift(i)
        df[f'events_lag_{i}'] = df['events'].shift(i)
    return df


lagged_data = create_lag_features(data)
# lagged_data = lagged_data.dropna()


X = lagged_data.drop(columns=['pitch_type'])
y = lagged_data['pitch_type']

X.head()


,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,...,arm_angle,pitch_type_lag_1,release_speed_lag_1,events_lag_1,pitch_type_lag_2,release_speed_lag_2,events_lag_2,pitch_type_lag_3,release_speed_lag_3,events_lag_3
2689,2022-05-08,97.0,-2.91,6.08,"Rainey, Tanner",543685,663432,single,hit_into_play,<NA>,...,36.5,None,<NA>,None,None,<NA>,None,None,<NA>,None
2764,2022-05-08,97.4,-2.72,6.12,"Rainey, Tanner",543685,663432,NaN,ball,<NA>,...,40.1,FF,97.0,single,None,<NA>,None,None,<NA>,None
2789,2022-05-08,87.9,-2.93,6.02,"Rainey, Tanner",660271,663432,double,hit_into_play,<NA>,...,38.2,FF,97.4,NaN,FF,97.0,single,None,<NA>,None
2893,2022-05-08,96.1,-3.03,6.05,"Rainey, Tanner",660271,663432,NaN,swinging_strike,<NA>,...,36.0,SL,87.9,double,FF,97.4,NaN,FF,97.0,single
2997,2022-05-08,97.1,-2.96,6.14,"Rainey, Tanner",545361,663432,strikeout,swinging_strike,<NA>,...,42.1,FF,96.1,NaN,SL,87.9,double,FF,97.4,NaN


In [ ]:
y.head()

,pitch_type
2689,FF
2764,FF
2789,SL
2893,FF
2997,FF


In [ ]:
from pybaseball import  playerid_lookup
from pybaseball import  statcast_pitcher
pitcher_stats = statcast_pitcher('2022-06-01', '2022-07-01', 663432)

Gathering Player Data


In [ ]:
pitcher_stats.groupby("pitch_type").release_speed.agg("mean")

,release_speed
pitch_type,
FF,97.558993
SL,89.069231


Test 2

In [ ]:
import pandas as pd

# Load the CSV file
data = pd.read_csv('combined_data.csv')

# Inspect the data
print(data.head())
print(data.columns)  # List all available columns

data = data.dropna(subset=['pitch_type'])
print(data.isnull().sum().sort_values(ascending=False))


  pitch_type   game_date  release_speed  release_pos_x  release_pos_z  \
0         SL  2018-10-28           84.0           3.05           5.26   
1         FF  2018-10-28           95.3           3.17           5.50   
2         FF  2018-10-28           96.4           3.07           5.54   
3         CH  2018-10-28           86.6           3.10           4.98   
4         SL  2018-10-28           78.5           3.31           5.16   

   player_name  batter  pitcher     events      description  ...  \
0  Sale, Chris  592518   519242  strikeout  swinging_strike  ...   
1  Sale, Chris  592518   519242        NaN             ball  ...   
2  Sale, Chris  592518   519242        NaN             foul  ...   
3  Sale, Chris  592518   519242        NaN  swinging_strike  ...   
4  Sale, Chris  571771   519242  strikeout  swinging_strike  ...   

   n_thruorder_pitcher  n_priorpa_thisgame_player_at_bat  \
0                    1                                 3   
1                    1          

In [ ]:
# Calculate the percentage of missing values per column
missing_percentage = data.isnull().mean() * 100
high_nan_columns = missing_percentage[missing_percentage > 50]
print(high_nan_columns)

columns_with_all_missing = data.columns[data.isnull().mean() == 1]
data = data.drop(columns=columns_with_all_missing)


events                              74.474187
spin_dir                           100.000000
spin_rate_deprecated               100.000000
break_angle_deprecated             100.000000
break_length_deprecated            100.000000
hit_location                        77.749176
bb_type                             82.754927
on_3b                               90.686276
on_2b                               81.340625
on_1b                               69.285189
hc_x                                82.836157
hc_y                                82.836157
tfs_deprecated                     100.000000
tfs_zulu_deprecated                100.000000
umpire                             100.000000
sv_id                              100.000000
hit_distance_sc                     69.402789
launch_speed                        69.128876
launch_angle                        69.112198
estimated_ba_using_speedangle       82.852074
estimated_woba_using_speedangle     75.327911
woba_value                        

In [ ]:
# data = data.drop(columns=high_nan_columns.index)
print(data.isnull().sum())

pitch_type                           0
game_date                            0
release_speed                       43
release_pos_x                      253
release_pos_z                      253
                                ...   
batter_days_until_next_game     127991
api_break_z_with_gravity            63
api_break_x_arm                     65
api_break_x_batter_in               65
arm_angle                      1262799
Length: 103, dtype: int64


In [ ]:
print(data.iloc[0].to_string())


pitch_type                                                           SL
game_date                                                    2018-10-28
release_speed                                                      84.0
release_pos_x                                                      3.05
release_pos_z                                                      5.26
player_name                                                 Sale, Chris
batter                                                           592518
pitcher                                                          519242
events                                                        strikeout
description                                             swinging_strike
zone                                                               13.0
des                                 Manny Machado strikes out swinging.
game_type                                                             W
stand                                                           

In [ ]:
# Replace NaN values appropriately
for col in data.columns:
    if data[col].dtype == 'object':  # For categorical columns (strings)
        data[col].fillna("UNKNOWN", inplace=True)
    elif pd.api.types.is_numeric_dtype(data[col]):
        data[col].fillna(-1, inplace=True)


<ipython-input-5-709f44068249>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna("UNKNOWN", inplace=True)
<ipython-input-5-709f44068249>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.met

In [ ]:
print(data.isnull().sum().sort_values(ascending=False))

pitch_type            0
launch_speed_angle    0
post_bat_score        0
post_home_score       0
post_away_score       0
                     ..
inning                0
outs_when_up          0
on_1b                 0
on_2b                 0
arm_angle             0
Length: 103, dtype: int64


In [ ]:
def create_lagged_features(data, lagged_columns, lags=3):

    for lag in range(1, lags + 1):
        for col in lagged_columns:
            if col in data.columns:  # Ensure column exists in the DataFrame
                data[f'{col}_lag_{lag}'] = data.groupby('game_pk')[col].shift(lag)
    return data

# List of columns to lag
lagged_columns = [
    'pitch_type', 'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z',
    'events', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'vx0', 'vy0', 'vz0',
    'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'effective_speed', 'spin_axis',
    'api_break_z_with_gravity', 'api_break_x_arm', 'api_break_x_batter_in', 'arm_angle'
]

# Create lagged features
data = create_lagged_features(data, lagged_columns, lags=3)

# Verify that lagged features have been added
print(data.head())


  pitch_type   game_date  release_speed  release_pos_x  release_pos_z  \
0         SL  2018-10-28           84.0           3.05           5.26   
1         FF  2018-10-28           95.3           3.17           5.50   
2         FF  2018-10-28           96.4           3.07           5.54   
3         CH  2018-10-28           86.6           3.10           4.98   
4         SL  2018-10-28           78.5           3.31           5.16   

   player_name  batter  pitcher     events      description  ...   ay_lag_3  \
0  Sale, Chris  592518   519242  strikeout  swinging_strike  ...        NaN   
1  Sale, Chris  592518   519242    UNKNOWN             ball  ...        NaN   
2  Sale, Chris  592518   519242    UNKNOWN             foul  ...        NaN   
3  Sale, Chris  592518   519242    UNKNOWN  swinging_strike  ...  26.649434   
4  Sale, Chris  571771   519242  strikeout  swinging_strike  ...  33.319856   

    az_lag_3 sz_top_lag_3 sz_bot_lag_3 effective_speed_lag_3 spin_axis_lag_3  \
0     

In [ ]:
data = data.dropna()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Identify non-numerical columns
non_numerical_columns = data.select_dtypes(include=['object']).columns

print(f"Non-numerical columns: {list(non_numerical_columns)}")

# Initialize a dictionary to store the LabelEncoders for each column
label_encoders = {}

# Label encode each non-numerical column
for col in non_numerical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Save the encoder for future use if needed

print("Label encoding completed.")


Non-numerical columns: ['pitch_type', 'game_date', 'player_name', 'events', 'description', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'bb_type', 'inning_topbot', 'pitch_name', 'if_fielding_alignment', 'of_fielding_alignment', 'pitch_type_lag_1', 'events_lag_1', 'pitch_type_lag_2', 'events_lag_2', 'pitch_type_lag_3', 'events_lag_3']
Label encoding completed.


In [ ]:
# List of columns for which lagged features were created
lagged_columns = [
    'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z', 'release_spin_rate', 'release_extension'
    'events', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'vx0', 'vy0', 'vz0', 'description',
    'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'effective_speed', 'spin_axis',
    'api_break_z_with_gravity', 'api_break_x_arm', 'api_break_x_batter_in', 'arm_angle'
]

# Remove these columns from the DataFrame (except 'pitch_type')
data = data.drop(columns=[col for col in lagged_columns if col in data.columns])

# Calculate correlation coefficients with 'pitch_type'
correlations = data.corr()['pitch_type'].sort_values(ascending=False)

# Display the correlation coefficients
print("Correlation Coefficients with 'pitch_type':")
print(correlations)

In [ ]:
print(correlations.to_string())

In [ ]:
# Select relevant columns
relevant_columns = [
    'game_pk', 'pitch_type', 'release_speed', 'events',
    'inning', 'inning_topbot', 'balls', 'strikes'
]
data = data[relevant_columns]
# Sort by game identifier and play order
data = data.sort_values(by=['game_pk', 'inning', 'inning_topbot']).reset_index(drop=True)


In [ ]:
def create_lag_features(df, lag=3):
    for i in range(1, lag + 1):
        # Lag features within each game
        df[f'pitch_type_lag_{i}'] = df.groupby('game_pk')['pitch_type'].shift(i)
        df[f'release_speed_lag_{i}'] = df.groupby('game_pk')['release_speed'].shift(i)
        df[f'events_lag_{i}'] = df.groupby('game_pk')['events'].shift(i)
    return df

# Apply lag feature creation
data = create_lag_features(data)

# Drop rows with NaN values caused by lagging
data = data.dropna()

# Inspect the lagged data
print(data.head())


       game_pk pitch_type  release_speed     events  inning inning_topbot  \
299     630099         FF           95.0  strikeout       9           Bot   
2213    630116         SI           92.5  field_out       6           Bot   
7335    630225         FF           97.0  field_out       7           Top   
11383   630313         SL           85.8  strikeout       5           Bot   
12091   630322         SL           81.7     single       6           Bot   

       balls  strikes pitch_type_lag_1  release_speed_lag_1 events_lag_1  \
299        0        2               FF                 89.1     home_run   
2213       1        2               SI                 91.8       single   
7335       3        2               SL                 88.7    field_out   
11383      3        2               CU                 75.0     home_run   
12091      1        2               FF                 92.6    force_out   

      pitch_type_lag_2  release_speed_lag_2 events_lag_2 pitch_type_lag_3  \
299

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoders for all categorical columns
categorical_columns = [
    'inning_topbot',
    'pitch_type_lag_1', 'pitch_type_lag_2', 'pitch_type_lag_3',
    'events_lag_1', 'events_lag_2', 'events_lag_3'
]

label_encoders = {}

# Encode categorical features
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])  # Encode entire dataset
    label_encoders[col] = le

# Encode target column (pitch_type)
target_encoder = LabelEncoder()
data['pitch_type'] = target_encoder.fit_transform(data['pitch_type'])


In [ ]:
# Exclude current pitch columns from features
excluded_columns = ['pitch_type', 'release_speed', 'events']
X = data.drop(columns=excluded_columns + ['game_pk'])  # Remove target and game_pk
y = data['pitch_type']  # Target column remains the current pitch type

# Inspect the final feature set
print(X.head())
print(y.head())


NameError: name 'data' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

# Split by unique game identifiers
game_ids = data['game_pk'].unique()
train_games, test_games = train_test_split(game_ids, test_size=0.2, random_state=42)

# Filter rows for training and testing sets based on game_pk
train_data = data[data['game_pk'].isin(train_games)]
test_data = data[data['game_pk'].isin(test_games)]

# Define features and target for train and test sets
X_train = train_data.drop(columns=excluded_columns + ['game_pk'])
y_train = train_data['pitch_type']
X_test = test_data.drop(columns=excluded_columns + ['game_pk'])
y_test = test_data['pitch_type']

# Check shapes of the training and testing sets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)


Training set shape: (500, 13)
Testing set shape: (124, 13)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")


Model Accuracy: 0.30


In [ ]:
from xgboost import XGBClassifier

# Train an XGBoost model
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate the model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Model Accuracy: {accuracy_xgb:.2f}")



XGBoost Model Accuracy: 0.31


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Randomized search for Random Forest
rf_random = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42),
                               param_distributions=param_grid,
                               n_iter=50, cv=3, random_state=42, n_jobs=-1)
rf_random.fit(X_train, y_train)

# Evaluate the best model
best_rf = rf_random.best_estimator_
y_pred_rf = best_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Tuned Random Forest Accuracy: {accuracy_rf:.2f}")


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Tuned Random Forest Accuracy: 0.37


In [ ]:
from sklearn.neural_network import MLPClassifier

# Train an MLPClassifier
mlp_model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
mlp_model.fit(X_train, y_train)

# Predict on the test set
y_pred_mlp = mlp_model.predict(X_test)

# Evaluate the model
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print(f"MLP Neural Network Accuracy: {accuracy_mlp:.2f}")


MLP Neural Network Accuracy: 0.32


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a model on scaled data
svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred_svm = svm_model.predict(X_test_scaled)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Model Accuracy: {accuracy_svm:.2f}")


SVM Model Accuracy: 0.31


test 3 lag 1

In [ ]:
def create_lag_features_with_fill(df, lags=3, num_fill_value=0, cat_fill_value="UNKNOWN"):

    lagged_columns = ['pitch_type', 'release_speed', 'spin_axis', 'events']  # Columns to lag
    categorical_columns = ['pitch_type', 'events']  # Specify which are categorical

    for lag in range(1, lags + 1):  # Loop through each lag
        for col in lagged_columns:
            if col in df.columns:  # Ensure column exists before lagging
                lag_col_name = f'{col}_lag_{lag}'
                df[lag_col_name] = df.groupby('game_pk')[col].shift(lag)

                # Handle NaN differently for categorical and numerical columns
                if col in categorical_columns:
                    df[lag_col_name] = df[lag_col_name].fillna(cat_fill_value)
                else:
                    df[lag_col_name] = df[lag_col_name].fillna(num_fill_value)
    return df

# Create lagged features and fill NaN values with a default
data = create_lag_features_with_fill(data, lags=1, num_fill_value=0, cat_fill_value="UNKNOWN")

# Check that no NaN values remain in lagged columns
print(data.isnull().sum())



pitch_type              60105
game_date                   0
release_speed           60145
release_pos_x           60355
release_pos_z           60355
                        ...  
arm_angle              214550
pitch_type_lag_1            0
release_speed_lag_1         0
spin_axis_lag_1             0
events_lag_1                0
Length: 117, dtype: int64


In [ ]:
selected_features = [
    # Game context
    'inning', 'inning_topbot', 'balls', 'strikes', 'outs_when_up',
    'on_1b', 'on_2b', 'on_3b', 'bat_score', 'fld_score',

    # Player info
    'p_throws', 'stand', 'player_name', 'batter', 'pitcher',

    # Lagged features
    'pitch_type_lag_1', 'release_speed_lag_1', 'spin_axis_lag_1', 'events_lag_1',

    # Target
    'pitch_type'
]

# Select only the relevant columns
data = data[selected_features]


In [ ]:
data = data.dropna()


In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = [
    'inning_topbot', 'p_throws', 'stand', 'player_name', 'batter', 'pitcher',
    'pitch_type_lag_1', 'events_lag_1'
]

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Encode the target column
target_encoder = LabelEncoder()
data['pitch_type'] = target_encoder.fit_transform(data['pitch_type'])


In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['pitch_type'])
y = data['pitch_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train the XGBoost model
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=target_encoder.classes_))


Model Accuracy: 0.46
              precision    recall  f1-score   support

          CH       0.32      0.20      0.25       965
          CS       0.00      0.00      0.00         4
          CU       0.35      0.19      0.24       662
          EP       0.50      0.60      0.55         5
          FA       0.94      0.74      0.83        42
          FC       0.42      0.29      0.34       728
          FF       0.48      0.68      0.56      3452
          FO       0.80      0.80      0.80         5
          FS       0.36      0.30      0.33       171
          KC       0.44      0.25      0.32       213
          SC       0.00      0.00      0.00         1
          SI       0.49      0.49      0.49      1671
          SL       0.47      0.40      0.43      1972
          ST       0.47      0.39      0.42       346
          SV       0.41      0.28      0.33        25

    accuracy                           0.46     10262
   macro avg       0.43      0.37      0.39     10262
weigh

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Randomized search for Random Forest
rf_random = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42),
                               param_distributions=param_grid,
                               n_iter=50, cv=3, random_state=42, n_jobs=-1)
rf_random.fit(X_train, y_train)

# Evaluate the best model
best_rf = rf_random.best_estimator_
y_pred_rf = best_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Tuned Random Forest Accuracy: {accuracy_rf:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Tuned Random Forest Accuracy: 0.46


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# Get the predicted probabilities for each class
y_pred_probs = model.predict_proba(X_test)

# Get the top 2 predicted class indices for each instance
top_2_preds = np.argsort(y_pred_probs, axis=1)[:, -3:]

# Check if the true label is in the top 2 predictions
top_2_correct = [y_test.iloc[i] in top_2_preds[i] for i in range(len(y_test))]

# Calculate Top-2 Accuracy
top_2_accuracy = np.mean(top_2_correct)
print(f"Top-2 Accuracy: {top_2_accuracy:.2f}")


Top-2 Accuracy: 0.86


Test 5
lag 3

In [ ]:
def create_lag_features_with_fill(df, lags=3, num_fill_value=0, cat_fill_value="UNKNOWN"):

    lagged_columns = ['pitch_type', 'release_speed', 'spin_axis', 'events']  # Columns to lag
    categorical_columns = ['pitch_type', 'events']  # Specify which are categorical

    for lag in range(1, lags + 1):  # Loop through each lag
        for col in lagged_columns:
            if col in df.columns:  # Ensure column exists before lagging
                lag_col_name = f'{col}_lag_{lag}'
                df[lag_col_name] = df.groupby('game_pk')[col].shift(lag)

                # Handle NaN differently for categorical and numerical columns
                if col in categorical_columns:
                    df[lag_col_name] = df[lag_col_name].fillna(cat_fill_value)
                else:
                    df[lag_col_name] = df[lag_col_name].fillna(num_fill_value)
    return df

# Create lagged features and fill NaN values with a default
data = create_lag_features_with_fill(data, lags=3, num_fill_value=0, cat_fill_value="UNKNOWN")

# Check that no NaN values remain in lagged columns
print(data.isnull().sum())



pitch_type             60105
game_date                  0
release_speed          60145
release_pos_x          60355
release_pos_z          60355
                       ...  
events_lag_2               0
pitch_type_lag_3           0
release_speed_lag_3        0
spin_axis_lag_3            0
events_lag_3               0
Length: 125, dtype: int64


In [ ]:
selected_features = [
    # Game context
    'inning', 'inning_topbot', 'balls', 'strikes', 'outs_when_up',
    'on_1b', 'on_2b', 'on_3b', 'bat_score', 'fld_score',

    # Player info
    'p_throws', 'stand', 'player_name', 'batter', 'pitcher',

    # Lagged features
    'pitch_type_lag_1', 'release_speed_lag_1', 'spin_axis_lag_1', 'events_lag_1',
    'pitch_type_lag_2', 'release_speed_lag_2', 'spin_axis_lag_2', 'events_lag_2',
    'pitch_type_lag_3', 'release_speed_lag_3', 'spin_axis_lag_3', 'events_lag_3',

    # Target
    'pitch_type'
]

# Select only the relevant columns
data = data[selected_features]


In [ ]:
data = data.dropna()


In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = [
    'inning_topbot', 'p_throws', 'stand', 'player_name', 'batter', 'pitcher',
    'pitch_type_lag_1', 'events_lag_1', 'pitch_type_lag_2', 'events_lag_2', 'pitch_type_lag_3', 'events_lag_3'
]

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Encode the target column
target_encoder = LabelEncoder()
data['pitch_type'] = target_encoder.fit_transform(data['pitch_type'])


<ipython-input-36-a21a64864855>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
<ipython-input-36-a21a64864855>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
<ipython-input-36-a21a64864855>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['pitch_type'])
y = data['pitch_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check shapes of the training and testing sets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)




Training set shape: (41044, 27)
Testing set shape: (10262, 27)


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train the XGBoost model
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=target_encoder.classes_))


Model Accuracy: 0.46
              precision    recall  f1-score   support

          CH       0.33      0.20      0.25       965
          CS       0.00      0.00      0.00         4
          CU       0.34      0.19      0.24       662
          EP       0.60      0.60      0.60         5
          FA       0.89      0.79      0.84        42
          FC       0.40      0.31      0.35       728
          FF       0.48      0.66      0.56      3452
          FO       0.60      0.60      0.60         5
          FS       0.40      0.29      0.33       171
          KC       0.50      0.28      0.36       213
          SC       0.00      0.00      0.00         1
          SI       0.49      0.51      0.50      1671
          SL       0.45      0.39      0.42      1972
          ST       0.47      0.40      0.43       346
          SV       0.44      0.32      0.37        25

    accuracy                           0.46     10262
   macro avg       0.43      0.37      0.39     10262
weigh

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Randomized search for Random Forest
rf_random = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42),
                               param_distributions=param_grid,
                               n_iter=50, cv=3, random_state=42, n_jobs=-1)
rf_random.fit(X_train, y_train)

# Evaluate the best model
best_rf = rf_random.best_estimator_
y_pred_rf = best_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Tuned Random Forest Accuracy: {accuracy_rf:.2f}")

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Tuned Random Forest Accuracy: 0.44


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# Get the predicted probabilities for each class
y_pred_probs = model.predict_proba(X_test)

# Get the top 2 predicted class indices for each instance
top_2_preds = np.argsort(y_pred_probs, axis=1)[:, -3:]

# Check if the true label is in the top 2 predictions
top_2_correct = [y_test.iloc[i] in top_2_preds[i] for i in range(len(y_test))]

# Calculate Top-2 Accuracy
top_2_accuracy = np.mean(top_2_correct)
print(f"Top-2 Accuracy: {top_2_accuracy:.2f}")


Top-2 Accuracy: 0.86


*Test* 6
lag 5

In [ ]:
def create_lag_features_with_fill(df, lags=3, num_fill_value=0, cat_fill_value="UNKNOWN"):

    lagged_columns = ['pitch_type', 'release_speed', 'spin_axis', 'events']  # Columns to lag
    categorical_columns = ['pitch_type', 'events']  # Specify which are categorical

    for lag in range(1, lags + 1):  # Loop through each lag
        for col in lagged_columns:
            if col in df.columns:  # Ensure column exists before lagging
                lag_col_name = f'{col}_lag_{lag}'
                df[lag_col_name] = df.groupby('game_pk')[col].shift(lag)

                # Handle NaN differently for categorical and numerical columns
                if col in categorical_columns:
                    df[lag_col_name] = df[lag_col_name].fillna(cat_fill_value)
                else:
                    df[lag_col_name] = df[lag_col_name].fillna(num_fill_value)
    return df

# Create lagged features and fill NaN values with a default
data = create_lag_features_with_fill(data, lags=5, num_fill_value=-1, cat_fill_value="UNKNOWN")

# Check that no NaN values remain in lagged columns
print(data.isnull().sum())




pitch_type             60105
game_date                  0
release_speed          60145
release_pos_x          60355
release_pos_z          60355
                       ...  
events_lag_4               0
pitch_type_lag_5           0
release_speed_lag_5        0
spin_axis_lag_5            0
events_lag_5               0
Length: 133, dtype: int64


In [ ]:
selected_features = [
    # Game context
    'inning', 'inning_topbot', 'balls', 'strikes', 'outs_when_up',
    'on_1b', 'on_2b', 'on_3b', 'bat_score', 'fld_score',

    # Player info
    'p_throws', 'stand', 'player_name', 'batter', 'pitcher',

    # Lagged features
    'pitch_type_lag_1', 'release_speed_lag_1', 'spin_axis_lag_1', 'events_lag_1',
    'pitch_type_lag_2', 'release_speed_lag_2', 'spin_axis_lag_2', 'events_lag_2',
    'pitch_type_lag_3', 'release_speed_lag_3', 'spin_axis_lag_3', 'events_lag_3',
    'pitch_type_lag_4', 'release_speed_lag_4', 'spin_axis_lag_4', 'events_lag_4',
    'pitch_type_lag_5', 'release_speed_lag_5', 'spin_axis_lag_5', 'events_lag_5',

    # Target
    'pitch_type'
]

# Select only the relevant columns
data = data[selected_features]


In [ ]:
data = data.dropna()


In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = [
    'inning_topbot', 'p_throws', 'stand', 'player_name', 'batter', 'pitcher',
    'pitch_type_lag_1', 'events_lag_1', 'pitch_type_lag_2', 'events_lag_2', 'pitch_type_lag_3', 'events_lag_3',
    'pitch_type_lag_4', 'events_lag_4', 'pitch_type_lag_5', 'events_lag_5'
]

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Encode the target column
target_encoder = LabelEncoder()
data['pitch_type'] = target_encoder.fit_transform(data['pitch_type'])


In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['pitch_type'])
y = data['pitch_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train the XGBoost model
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=target_encoder.classes_))


Model Accuracy: 0.46
              precision    recall  f1-score   support

          CH       0.32      0.19      0.24       965
          CS       0.00      0.00      0.00         4
          CU       0.35      0.19      0.24       662
          EP       0.60      0.60      0.60         5
          FA       0.91      0.74      0.82        42
          FC       0.41      0.32      0.36       728
          FF       0.48      0.67      0.56      3452
          FO       0.50      0.40      0.44         5
          FS       0.41      0.28      0.33       171
          KC       0.50      0.23      0.32       213
          SC       0.00      0.00      0.00         1
          SI       0.49      0.51      0.50      1671
          SL       0.44      0.38      0.41      1972
          ST       0.48      0.36      0.41       346
          SV       0.47      0.28      0.35        25

    accuracy                           0.46     10262
   macro avg       0.42      0.34      0.37     10262
weigh

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Randomized search for Random Forest
rf_random = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42),
                               param_distributions=param_grid,
                               n_iter=50, cv=3, random_state=42, n_jobs=-1)
rf_random.fit(X_train, y_train)

# Evaluate the best model
best_rf = rf_random.best_estimator_
y_pred_rf = best_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Tuned Random Forest Accuracy: {accuracy_rf:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Tuned Random Forest Accuracy: 0.46


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# Get the predicted probabilities for each class
y_pred_probs = model.predict_proba(X_test)

# Get the top 2 predicted class indices for each instance
top_2_preds = np.argsort(y_pred_probs, axis=1)[:, -3:]

# Check if the true label is in the top 2 predictions
top_2_correct = [y_test.iloc[i] in top_2_preds[i] for i in range(len(y_test))]

# Calculate Top-2 Accuracy
top_2_accuracy = np.mean(top_2_correct)
print(f"Top-2 Accuracy: {top_2_accuracy:.2f}")


Top-2 Accuracy: 0.86


In [ ]:
#check correlation coeff

In [ ]:
corr_matrix = data.corr()
print(corr_matrix['pitch_type'].sort_values(ascending=False))


pitch_type             1.000000
pitch_type_lag_1       0.301092
pitch_type_lag_2       0.236556
pitch_type_lag_3       0.201933
pitch_type_lag_4       0.155798
pitch_type_lag_5       0.130911
stand                  0.104877
inning                 0.069979
fld_score              0.030928
p_throws               0.028831
pitcher                0.019713
bat_score              0.013836
release_speed_lag_1    0.012815
on_1b                  0.012684
on_3b                  0.009397
events_lag_1           0.008748
release_speed_lag_2    0.005781
on_2b                  0.002688
events_lag_3           0.002519
events_lag_5           0.002254
events_lag_2           0.001543
release_speed_lag_3    0.000844
release_speed_lag_4   -0.002461
events_lag_4          -0.004468
balls                 -0.006858
inning_topbot         -0.007612
batter                -0.008840
release_speed_lag_5   -0.008846
outs_when_up          -0.011591
strikes               -0.012961
spin_axis_lag_5       -0.015506
spin_axi

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty='l2', C=0.1, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=target_encoder.classes_))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model Accuracy: 0.34


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

          CH       0.00      0.00      0.00       965
          CS       0.00      0.00      0.00         4
          CU       0.00      0.00      0.00       662
          EP       0.00      0.00      0.00         5
          FA       0.00      0.00      0.00        42
          FC       0.00      0.00      0.00       728
          FF       0.34      1.00      0.50      3452
          FO       0.00      0.00      0.00         5
          FS       0.00      0.00      0.00       171
          KC       0.00      0.00      0.00       213
          SC       0.00      0.00      0.00         1
          SI       0.00      0.00      0.00      1671
          SL       0.00      0.00      0.00      1972
          ST       0.00      0.00      0.00       346
          SV       0.00      0.00      0.00        25

    accuracy                           0.34     10262
   macro avg       0.02      0.07      0.03     10262
weighted avg       0.11   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert data to NumPy arrays
X = data.drop(columns=['pitch_type']).values
y = data['pitch_type'].values

# Standardize numerical features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Define the neural network
class PitchTypeNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(PitchTypeNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
from torch.optim import Adam

# Model setup
input_size = X_train.shape[1]
num_classes = len(data['pitch_type'].unique())
model = PitchTypeNN(input_size, num_classes)

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")


Epoch 1/1000, Loss: 1.8188
Epoch 2/1000, Loss: 1.7095
Epoch 3/1000, Loss: 1.6790
Epoch 4/1000, Loss: 1.6620
Epoch 5/1000, Loss: 1.6486
Epoch 6/1000, Loss: 1.6394
Epoch 7/1000, Loss: 1.6333
Epoch 8/1000, Loss: 1.6268
Epoch 9/1000, Loss: 1.6225
Epoch 10/1000, Loss: 1.6157
Epoch 11/1000, Loss: 1.6121
Epoch 12/1000, Loss: 1.6077
Epoch 13/1000, Loss: 1.6052
Epoch 14/1000, Loss: 1.6036
Epoch 15/1000, Loss: 1.5996
Epoch 16/1000, Loss: 1.5959
Epoch 17/1000, Loss: 1.5927
Epoch 18/1000, Loss: 1.5920
Epoch 19/1000, Loss: 1.5874
Epoch 20/1000, Loss: 1.5851
Epoch 21/1000, Loss: 1.5822
Epoch 22/1000, Loss: 1.5821
Epoch 23/1000, Loss: 1.5783
Epoch 24/1000, Loss: 1.5783
Epoch 25/1000, Loss: 1.5744
Epoch 26/1000, Loss: 1.5721
Epoch 27/1000, Loss: 1.5715
Epoch 28/1000, Loss: 1.5666
Epoch 29/1000, Loss: 1.5686
Epoch 30/1000, Loss: 1.5651
Epoch 31/1000, Loss: 1.5654
Epoch 32/1000, Loss: 1.5626
Epoch 33/1000, Loss: 1.5623
Epoch 34/1000, Loss: 1.5628
Epoch 35/1000, Loss: 1.5579
Epoch 36/1000, Loss: 1.5569
E

In [ ]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.4277
